In [ ]:
 #@title AUTO RECONNECT Colab Runtime to prevent notebook from disconnecting
 
import IPython
from google.colab import output
 
display(IPython.display.Javascript('''
function ClickConnect(){
  btn = document.querySelector("colab-connect-button")
  if (btn != null){
    console.log("Click colab-connect-button");
    btn.click()
    }
    
  btn = document.getElementById('ok')
  if (btn != null){
    console.log("Click reconnect");
    btn.click()
    }
  }
    
setInterval(ClickConnect,60000)
'''))

In [ ]:
#@title Mount/Unmount Google Drive
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

In [ ]:
#@title aria2</> Direct link downloader
#@markdown Auto share link supported : Disk.yandex.com, Gdrive and youtube-dl all extractor...
URL = "" #@param {type:"string"}
#@markdown <center><h5>Default output path /content/downloads/</h5></center>
OUTPUT_PATH = "" #@param {type:"string"}

import pathlib
import shutil
import hashlib
import requests
from urllib.parse import urlparse
from os import path, mkdir
if not path.exists("/root/.ipython/ocr.py"): 
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

def youtubedlInstall():
  if not path.isfile("/usr/local/bin/youtube-dl"):
    cmdC = "rm -rf /content/sample_data/ && " \
            " mkdir -p -m 666 /root/.YouTube-DL/ &&" \
            " apt-get install atomicparsley &&" \
            " curl -L https://yt-dl.org/downloads/latest/youtube-dl " \
            "-o /usr/local/bin/youtube-dl &&" \
            " chmod a+rx /usr/local/bin/youtube-dl"
    get_ipython().system_raw(cmdC)

def aria2Install():
  runSh('apt install -y aria2')

def istmd(URL): 
  link = urlparse(URL)
    
  #YandexDisk
  if link.netloc == "yadi.sk":
    API_ENDPOINT = 'https://cloud-api.yandex.net/v1/disk/public/resources/' \
                    '?public_key={}&path=/{}&offset={}'
    dry = False
    def md5sum(file_path):
        md5 = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
                md5.update(chunk)
        return md5.hexdigest()


    def check_and_download_file(target_path, url, size, checksum):
        if path.isfile(target_path):
            if size == path.getsize(target_path):
                if checksum == md5sum(target_path):
                    print('URL {}'.format(url))
                    print('skipping correct {}'.format(target_path))
                    return
        if not dry:
            print('URL {}'.format(url))
            print('downloading {}'.format(target_path))
            runSh(f'aria2c -x 16 -s 16 -k 1M -d {OUTPUT_PATH} {url}', output=True)
            # r = requests.get(url, stream=True)
            # with open(target_path, 'wb') as f:
            #     shutil.copyfileobj(r.raw, f)


    def download_path(target_path, public_key, source_path, offset=0):
        print('getting "{}" at offset {}'.format(source_path, offset))
        current_path = path.join(target_path, source_path)
        pathlib.Path(current_path).mkdir(parents=True, exist_ok=True)
        jsn = requests.get(API_ENDPOINT.format(public_key, source_path, offset)).json()
        def try_as_file(j):
            if 'file' in j:
                file_save_path = path.join(current_path, j['name'])
                check_and_download_file(file_save_path, j['file'], j['size'], j['md5'])
                return True
            return False

        # first try to treat the actual json as a single file description
        if try_as_file(jsn):
            return

        # otherwise treat it as a directory
        emb = jsn['_embedded']
        items = emb['items']
        for i in items:
            # each item can be a file...
            if try_as_file(i):
                continue
            # ... or a directory
            else:
                subdir_path = path.join(source_path, i['name'])
                download_path(target_path, public_key, subdir_path)

        # check if current directory has more items
        last = offset + emb['limit']
        if last < emb['total']:
            download_path(target_path, public_key, source_path, last)
    download_path(OUTPUT_PATH, URL, '')
    return False  
  return URL

if not OUTPUT_PATH:
  OUTPUT_PATH = "/content/downloads/"
  
if not URL == "":
  aria2Install()
  youtubedlInstall()
  try:
    mkdir("downloads")
  except FileExistsError:
    pass
  url = istmd(URL)
  if url != False:
    print('URL {}'.format(URL))
    cmdC = f'youtube-dl -o "{OUTPUT_PATH}/%(title)s" {URL} ' \
            '--external-downloader aria2c ' \
            '--external-downloader-args "-x 16 -s 16 -k 1M"'
    runSh(cmdC, output=True)
else:
  print("Please input url")

In [ ]:
# ============================= FORM ============================= #
#@title <h3>⬅️ Install Cloud Commander (file manager)</h3>
#@markdown <br><center><img src='https://biplobsd.github.io/RLabClone/img/title_cloud_commander.png' height="60" alt="netdata"/></center>
# ================================================================ #
USE_FREE_TOKEN = False  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "AU" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]

import os, psutil, time, urllib.request
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    loadingAn,
    PortForward_wrapper,
)

loadingAn()
if os.path.isfile("/tools/node/bin/cloudcmd") == False:
  get_ipython().system_raw("npm cache clean -f && npm install -g n && n stable && npm i cloudcmd -g --force")

try:
  urllib.request.urlopen('http://localhost:7007')
except urllib.error.URLError:
  !nohup cloudcmd --online --no-auth --show-config --show-file-name \
        --editor 'deepword' --packer 'tar' --port 7007 \
        --no-confirm-copy --confirm-move --name 'File Manager' \
        --keys-panel --no-contact --console --sync-console-path \
        --no-terminal --no-vim --columns 'name-size-date' --no-log &
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "localhost" #@param ["ngrok", "localhost"]
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['netdata', 19999, 'http'], 
                            ['cloudcmd', 7007, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/netdataCloudcmd.yml", 7044]
).start('cloudcmd')

#Make sure to use GPU hardware accelerator

1. Clone the repository (https://github.com/rokibulislaam/colab-ffmpeg-cuda)


In [1]:
!git clone https://github.com/rokibulislaam/colab-ffmpeg-cuda.git

Cloning into 'colab-ffmpeg-cuda'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 59 (delta 24), reused 39 (delta 15), pack-reused 0
Unpacking objects: 100% (59/59), done.
Checking out files: 100% (32/32), done.


Copy all the pre-built binaries from ./colab-ffmpeg-cuda/bin/ to /usr/bin/ (Recommended)

In [2]:
!cp -r ./colab-ffmpeg-cuda/bin/. /usr/bin/

Check the installed ffmpeg version

In [3]:
!ffmpeg -version

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
configuration: --enable-cuda-nvcc --enable-cuvid --enable-nvenc --enable-libnpp --enable-cuda-llvm --pkgconfigdir=/content/workspace/lib/pkgconfig --prefix=/content/workspace --pkg-config-flags=--static --extra-cflags='-I/content/workspace/include -I/usr/local/cuda/include' --extra-ldflags='-L/content/workspace/lib -L/usr/local/cuda/lib64' --extra-libs='-lpthread -lm' --enable-static --disable-debug --disable-shared --disable-ffplay --disable-doc --enable-openssl --enable-gpl --enable-version3 --enable-nonfree --enable-pthreads --enable-libvpx --enable-libmp3lame --enable-libopus --enable-libtheora --enable-libvorbis --enable-libx264 --enable-libx265 --enable-runtime-cpudetect --enable-libfdk-aac --enable-avfilter --enable-libopencore_amrwb --enable-libopencore_amrnb --enable-filters --enable-libvidstab --enable-libaom --enable-libsrt
libavutil      56. 51.100 / 56. 51.100


In [ ]:
#@title Download Any Video with FFmpeg
URL = ""# @param{type:"string"}
#@markdown Don't Add Slash / at the end of Path
Saving_Path = ""# @param{type:"string"}
File_Name = ""# @param{type:"string"}
#@markdown Major Extensions are given ⬇️ (But you can Still Type your custom)
Extension = ".mp4" #@param [".mp4", ".mkv", ".avi", ".flv", ".wmv", ".mov"] {allow-input: true}
!ffmpeg -i "$URL" -bsf:a aac_adtstoasc -vcodec copy -c copy -crf 17 "$Saving_Path/$File_Name$Extension"

In [ ]:
#@title FFmpeg for Trimming of Video
URL = ""# @param {type:"string"}
print("\n")
StartTime = ""# @param {type:"string"}
if (StartTime == ""):
  StartTime = "00:00:00"
EndTime = ""# @param {type:"string"}
print("\n")
#@markdown Don't Add Slash / at the end of Path
Saving_Path = ""# @param {type:"string"}
File_Name = "" #@param {type:"string"}
#@markdown Major Extensions are given ⬇️ (But you can Still Type your custom)
Extension = ".mp4" #@param [".mp4", ".mkv", ".avi", ".flv", ".wmv", ".mov"] {allow-input: true}
!ffmpeg -i "$URL" -ss $StartTime -to $EndTime -c copy "$Saving_Path/(Trimmed) $File_Name$Extension"


In [ ]:
#@title FFmpeg for converting codec of video and audio

URL = ""# @param {type:"string"}
#@markdown Major Codec is already in Dropdown⬇️ (But you can Still Type your custom)
Video_Codec = "libx264" #@param ["libx264", "libx265", "nvenc_h264", "nvenc_hevc"] {allow-input: true}
Audio_Codec = "AAC" #@param ["AAC", "MP3"] {allow-input: true}
#@markdown Don't Add Slash / at the end of Path
Saving_Path = ""# @param {type:"string"}
File_Name = "" #@param {type:"string"}
#@markdown Major Extensions are given ⬇️ (But you can Still Type your custom)
Extension = ".mp4" #@param [".mp4", ".mkv", ".avi", ".flv", ".wmv", ".mov"] {allow-input: true}
!ffmpeg -i "$URL" -c:v $Video_Codec -c:a $Audio_Codec 


In [ ]:
#@title Custom FFmpeg Command
Command = ""# @param {type:"string"}
!$Command

# Is it not working?
If you are having trouble with the pre-built binaries, buid the binaries from scratch (It may take more than half an hour)

In [ ]:
!chmod +x ./colab-ffmpeg-cuda/build && ./colab-ffmpeg-cuda/build --build

There you go, ffmpeg with the required binaries should be installed to /usr/bin.
